In [1]:
import os,cv2,pytesseract,re
from PIL import Image
from fuzzywuzzy import fuzz
from pyzbar.pyzbar import decode
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [2]:
def get_text(imgfile):
    img = cv2.imread(imgfile)
    grayimg=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    tempfile = "{}.png".format(os.getpid())
    cv2.imwrite(tempfile, grayimg)
    text = pytesseract.image_to_string(Image.open(tempfile))
    os.remove(tempfile)
    return text

def getEno(text):
    x=re.findall('[0-9]{4}\/[0-9]{5}\/[0-9]{5}',text)
    if len(x)>0:
        return x[0]

def getPhone(text):
    x=re.findall(r'[0-9]{10}',text)
    return x[len(x)-1]

def getAno(text):
    return re.findall('[0-9]{4}[ ][0-9]{4}[ ][0-9]{4}',text)[0].replace(" ","")

def getDOB(text):
    x=re.findall('[0-9]{2}\/[0-9]{2}\/[0-9]{4}',text)
    return x[len(x)-1]

def getGender(text):
    if text.find('Male\n')==-1 and text.find('MALE\n')==-1 :
        return 'Female'
    else:
        return 'Male'

In [3]:
imgfile='aadhar.jpg'
text=get_text(imgfile)
aadhar_details={'Enrollment No':getEno(text),
'Phone':getPhone(text),
'Aadhar No':getAno(text),
'DOB':getDOB(text),
'Gender':getGender(text)}

aadhar_details

{'Enrollment No': '0000/00213/30165',
 'Phone': '9867615015',
 'Aadhar No': '220862372264',
 'DOB': '25/01/2001',
 'Gender': 'Male'}

In [4]:
def getQRAno(text):
    return re.findall('[0-9]{12}',text)[0]

def getQRDOB(text):
    x=re.findall('[0-9]{2}\/[0-9]{2}\/[0-9]{4}',text)
    return x[len(x)-1]

def getQRName(text):
    x=re.findall('[0-9]{2}\/[0-9]{2}\/[0-9]{4}',text)
    return x[len(x)-1]

def getQRGender(text):
    index=text.find('gender="')+len('gender="')
    if text[index]=='M':
        return 'Male'
    else:
        return 'Female'

In [5]:
qrtext=decode(Image.open('aadhar.jpg'))[0].data.decode('utf-8')

qr_details={'Aadhar No':getQRAno(qrtext),
'DOB':getQRDOB(qrtext),
'Gender':getQRGender(qrtext)}

qr_details

{'Aadhar No': '220862372264', 'DOB': '25/01/2001', 'Gender': 'Male'}

In [6]:
def validate(aadhar_details,qr_details):
    if fuzz.partial_ratio(aadhar_details['Aadhar No'],qr_details['Aadhar No'])<80:
        print("INVALID AADHAR NO!!")
    elif fuzz.partial_ratio(aadhar_details['DOB'],qr_details['DOB'])<80:
        print("INVALID DOB!!")
    elif fuzz.partial_ratio(aadhar_details['Gender'],qr_details['Gender'])<80:
        print("INVALID GENDER!!")
    else:
        print("Verified!!")
        
validate(aadhar_details,qr_details)

Verified!!
